# Custom Target Practice

## Model

In [1]:
] activate "/home/jaimerz/julia_repos/CMBLensingInferenceTestProblem"

  Activating project at `~/julia_repos/CMBLensingInferenceTestProblem`


In [2]:
# The statistical inference frame-work we will use
using MCHMC
using Interpolations
using ForwardDiff
using LinearAlgebra
using Distributions
using ForwardDiff
using PyPlot

In [3]:
fs8_zs = [0.38, 0.51, 0.61, 1.48, 0.44, 0.6, 0.73, 0.6, 0.86, 0.067, 1.4]
fs8_data = [0.49749, 0.457523, 0.436148, 0.462, 0.413, 0.39, 0.437, 0.55, 0.4, 0.423, 0.482]
fs8_cov = [0.00203355 0.000811829 0.000264615 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.000811829 0.00142289 0.000662824 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 
           0.000264615 0.000662824 0.00118576 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.002025 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0064 0.00257 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.00257 0.003969 0.00254 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.00254 0.005184 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0144 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0121 0.0 0.0; 
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.003025 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.013456000000000001];

In [4]:
function make_fs8(Ωm, σ8; Ωr=8.24*10^-5)
    # ODE solution for growth factor
    x_Dz = LinRange(0, log(1+1100), 300)
    dx_Dz = x_Dz[2]-x_Dz[1]
    z_Dz = @.(exp(x_Dz) - 1)
    a_Dz = @.(1/(1+z_Dz))
    aa = reverse(a_Dz)
    e =  @.(sqrt.(abs(Ωm)*(1+z_Dz)^3+Ωr*(1+z_Dz)^4+(1-Ωm-Ωr)))
    ee = reverse(e)

    dd = zeros(typeof(Ωm), 300)
    yy = zeros(typeof(Ωm), 300)
    dd[1] = aa[1]
    yy[1] = aa[1]^3*ee[end]

    for i in 1:(300-1)
        A0 = -1.5 * Ωm / (aa[i]*ee[i])
        B0 = -1. / (aa[i]^2*ee[i])
        A1 = -1.5 * Ωm / (aa[i+1]*ee[i+1])
        B1 = -1. / (aa[i+1]^2*ee[i+1])
        yy[i+1] = (1+0.5*dx_Dz^2*A0*B0)*yy[i] + 0.5*(A0+A1)*dx_Dz*dd[i]
        dd[i+1] = 0.5*(B0+B1)*dx_Dz*yy[i] + (1+0.5*dx_Dz^2*A0*B0)*dd[i]
    end

    y = reverse(yy)
    d = reverse(dd)

    Dzi = linear_interpolation(z_Dz, d./d[1], extrapolation_bc=Line())
    fs8zi = linear_interpolation(z_Dz, -σ8 .* y./ (a_Dz.^2 .*e.*d[1]),
                                 extrapolation_bc=Line())
    return fs8zi
end

make_fs8 (generic function with 1 method)

In [5]:
using Turing
@model function model(data; cov = fs8_cov) 
    # Define priors
    #KiDS priors
    Ωm ~ Normal(0.3, 0.1) # ~ Uniform(0.1, 0.9) 
    σ8 ~ Normal(0.8, 0.3)
    fs8_itp = make_fs8(Ωm, σ8)
    theory = fs8_itp(fs8_zs)
    data ~ MvNormal(theory, cov)
end;

In [6]:
priors = [Uniform(0.1, 0.4), Normal(0.8, 0.3)]

2-element Vector{Distribution{Univariate, Continuous}}:
 Uniform{Float64}(a=0.1, b=0.4)
 Normal{Float64}(μ=0.8, σ=0.3)

In [7]:
function nlogp(pars; priors=priors, data=fs8_data, inv_cov = inv(fs8_cov)) 
    # Define priors
    #KiDS priors
    Ωm, σ8 = pars
    fs8_itp = make_fs8(Ωm, σ8)
    theory = fs8_itp(fs8_zs)
    diff = abs.(theory.-data)
    Xi2 = dot(diff, inv_cov * diff)
    logprior = 0
    for (par, prior) in zip(pars, priors)
        logprior += logpdf(prior, par)
    end
    return -(-0.5 * Xi2 + logprior)
end;

function grad_nlogp(pars)
    return ForwardDiff.gradient(nlogp, pars)
end
    

grad_nlogp (generic function with 1 method)

## Sampling

In [14]:
target = CustomTarget(nlogp, grad_nlogp, priors);

In [15]:
eps=0.005
L=sqrt(2)
spl = Sampler(eps=eps, L=L);

In [16]:
samples = Sample(spl, target; num_steps=10000);

LoadError: UndefVarError: `linear_interpolation` not defined

## Plotting

In [12]:
Wms = [sample[1] for sample in samples]
s8s = [sample[2] for sample in samples];

LoadError: UndefVarError: `samples` not defined

In [13]:
plt.hist2d(Wms, s8s, bins=100, range=[[0.1, 0.4],[0.6, 1.2]])
plt.xlabel("Wm")
plt.ylabel("s8")
plt.title("Custom Target");

LoadError: UndefVarError: `Wms` not defined